In [1]:
from nlu_engine import Analytics
from nlu_engine import RenderJSON
from nlu_engine import DataUtils
from nlu_engine import MacroEntityRefinement
from nlu_engine import MacroDataRefinement
from nlu_engine import NLUEngine
import nltk
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from nlu_engine import EntityExtractor, crf

# Bug report

When running `cross_val_predict` from sklearn on the entity extractor for `general` domain, the following error is encountered:

```
The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.

Canceled future for execute_request message before replies were done
```

It's funny because if you run `cross_val_predict` on the entity extractor for any other domain, it works perfectly. Even funnier, if you run it on all of the domains together, it ALSO works! 

That doesn't seem logical to me. Why would it work on every domain separately except for `general` but it works when you run them together?

Let's load our data that causes the error.

In [2]:
nlu_data_df = pd.read_csv(
        'data/refined/nlu_data_refined_df.csv', sep=',', index_col=0)

nlu_data_df = DataUtils.convert_annotated_utterances_to_normalised_utterances(
    nlu_data_df)

removed_nlu_data_refined_df = nlu_data_df[nlu_data_df['remove'] != True]

domain = 'general'
domain_df = removed_nlu_data_refined_df[removed_nlu_data_refined_df['scenario'] == domain]


In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

If we run `cross_val_predict` on the entity extractor for all domains together, it works perfectly.

In [3]:
entity_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=removed_nlu_data_refined_df, cv=4)

Evaluating entity classifier
Cross validating with CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Time it took to cross validate CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100): 272.7438073158264


/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
X, y = EntityExtractor.get_targets_and_labels(domain_df)

Here is where it goes wrong...

In [ ]:
predictions = Analytics.cross_validate_classifier(crf, X, y, cv=3)